In [ ]:
from skimage import color, exposure

from skimage.feature import hog, corner_harris, corner_peaks # Import corner_harris and corner_peaks

from skimage import color, exposure

from skimage.feature import hog, corner_harris, corner_peaks # Import corner_harris and corner_peaks

histogram of pixel values

In [ ]:
# Get a batch of images and labels

images, labels = next(iter(train_loader))



# Select the first image from the batch

image = images[0]



# Convert the image from a PyTorch tensor to a NumPy array

image = image.numpy().transpose((1, 2, 0))



# Plot the original image

plt.figure(figsize=(12, 4))



plt.subplot(1, 2, 1)

plt.imshow(image)

plt.title(f"Label: {'Benign' if labels[3] == 0 else 'Malignant'}")



# Plot the histogram of pixel values

plt.subplot(1, 2, 2)

plt.hist(image.ravel(), bins=256, color='orange', alpha=0.7)

plt.title("Pixel Value Histogram")

plt.xlabel("Pixel Value")

plt.ylabel("Frequency")



plt.tight_layout()

plt.show()


Calculate GLCM properties

In [ ]:
import numpy as np

import matplotlib.pyplot as plt

from skimage.feature import graycomatrix, graycoprops  # Correct import

from skimage.color import rgb2gray

import torch



# Get a batch of images and labels

images, labels = next(iter(train_loader))



# Select the first image from the batch

image = images[0]



# Convert the image from a PyTorch tensor to a NumPy array and grayscale it

image = image.permute(1, 2, 0).cpu().numpy()  # Convert to HxWxC format and ensure it's on CPU

gray_image = rgb2gray(image)  # Convert RGB image to grayscale



# Plot the original image

plt.figure(figsize=(12, 6))



plt.subplot(1, 3, 1)

plt.imshow(image)

plt.title(f"Label: {'Benign' if labels[0].item() == 0 else 'Malignant'}")







# Calculate GLCM and GLCM properties

distances = [1]  # Pixel pair distance

angles = [0]  # Angle (0 degrees)

glcm = graycomatrix((gray_image * 255).astype(np.uint8), distances=distances, angles=angles, symmetric=True, normed=True)  # Using graycomatrix instead of greycomatrix



# Extract GLCM properties

contrast = graycoprops(glcm, 'contrast')[0, 0]  # Using graycoprops instead of greycoprops

correlation = graycoprops(glcm, 'correlation')[0, 0]  # Using graycoprops instead of greycoprops

energy = graycoprops(glcm, 'energy')[0, 0]  # Using graycoprops instead of greycoprops

homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]  # Using graycoprops instead of greycoprops



# Display GLCM properties

plt.subplot(1, 3, 3)

plt.text(0.1, 0.8, f'Contrast: {contrast:.4f}', fontsize=12)

plt.text(0.1, 0.6, f'Correlation: {correlation:.4f}', fontsize=12)

plt.text(0.1, 0.4, f'Energy: {energy:.4f}', fontsize=12)

plt.text(0.1, 0.2, f'Homogeneity: {homogeneity:.4f}', fontsize=12)

plt.axis('off')

plt.title("GLCM Properties")



plt.tight_layout()

plt.show()

HOG image for visualization

In [ ]:




image, labels = next(iter(train_loader))  # Get a batch of images and labels



image = image[5]

image = image.numpy().transpose((1, 2, 0))





# Convert the image to grayscale



gray_image = color.rgb2gray(image)



plt.figure(figsize=(12, 5))

# Compute HOG features and the HOG image for visualization

hog_features, hog_image = hog(

    gray_image,

    orientations=9,

    pixels_per_cell=(8, 8),

    cells_per_block=(2, 2),

    block_norm='L2-Hys',

    visualize=True,

    channel_axis=None

)



# Plot the original image and the HOG image

plt.figure(figsize=(12, 6))



plt.subplot(1, 2, 1)

plt.imshow(image, cmap='gray')

plt.title(f"Label: {'Benign' if labels[5] == 0 else 'Malignant'}")







plt.subplot(1, 2, 2)

plt.imshow(hog_image, cmap='gray')

plt.title("HOG Image")

#coner dectection

In [ ]:
#coner dectection

from skimage.feature import hog, corner_harris, corner_peaks # Import corner_harris and corner_peaks



image, labels = next(iter(train_loader))  # Get a batch of images and labels



image = image[5]

image = image.numpy().transpose((1, 2, 0))



gray_image = color.rgb2gray(image)



# Perform Harris corner detection

corners = corner_harris(gray_image)



# Perform corner peak detection

coords = corner_peaks(corners, min_distance=5)



# Plot the original image with detected corners

plt.figure(figsize=(12, 5))



plt.subplot(1, 2, 1)

plt.imshow(gray_image, cmap='gray')

plt.title('Original Image')

plt.axis('off')



plt.subplot(1, 2, 2)

plt.imshow(gray_image, cmap='gray')

plt.scatter(coords[:, 1], coords[:, 0], s=15, c='red', marker='o')

plt.title('Corner Detection')

plt.axis('off')



plt.tight_layout()

plt.show()



# Display number of corners detected

print(f'Number of corners detected: {len(coords)}')

Sobel operator

In [ ]:
import numpy as np

import matplotlib.pyplot as plt

from skimage import color, io

from numba import jit, prange



# Define Sobel operator kernels

sobel_x = np.array([[-1, 0, 1],

                    [-2, 0, 2],

                    [-1, 0, 1]])



sobel_y = np.array([[-1, -2, -1],

                    [0, 0, 0],

                    [1, 2, 1]])



# Define the custom convolution function with Numba for speedup

@jit(nopython=True, parallel=True)

def convolve(x, h):

    xh, xw = x.shape

    hh, hw = h.shape

    # Kernel radius

    rh, rw = np.array(h.shape)//2

    # Init output

    output = np.zeros(x.shape)

    for n1 in prange(rh, xh-rh):

        for n2 in prange(rw, xw-rw):

            value = 0

            for k1 in prange(hh):

                for k2 in prange(hw):

                    value += h[k1, k2]*x[n1 + k1 - rh, n2 + k2 - rw]

            output[n1, n2] = value

    return output



# Load the image

image, labels = next(iter(train_loader))  # Get a batch of images and labels



image = image[5]

image = image.numpy().transpose((1, 2, 0))

# Convert the image to grayscale

gray_image = color.rgb2gray(image)



# Apply the custom convolution with the Sobel operator in the x and y directions

gradient_x = convolve(gray_image, sobel_x)

gradient_y = convolve(gray_image, sobel_y)



# Combine the Sobel x and y results

sobel_combined = np.hypot(gradient_x, gradient_y)



# Normalize the combined result to range [0, 1]

sobel_combined /= sobel_combined.max()



# Plot the original image and the Sobel edge-detected images

plt.figure(figsize=(18, 5))



plt.subplot(1, 3, 1)

plt.imshow(gray_image, cmap='gray')

plt.title('Original Image')

plt.axis('off')



plt.subplot(1, 3, 2)

plt.imshow(np.abs(gradient_x), cmap='gray')

plt.title('Sobel - X Direction')

plt.axis('off')



plt.subplot(1, 3, 3)

plt.imshow(np.abs(gradient_y), cmap='gray')

plt.title('Sobel - Y Direction')

plt.axis('off')



plt.figure(figsize=(3, 3))

plt.imshow(sobel_combined, cmap='gray')

plt.title('Sobel Edge Detection (Combined)')

plt.axis('off')



plt.tight_layout()

plt.show()






In [ ]:
import numpy as np

import matplotlib.pyplot as plt

from skimage import color, io

from skimage.feature import local_binary_pattern



# Parameters for LBP

radius = 1  # Radius of the circle

n_points = 8 * radius  # Number of points considered for the LBP



# Load the image

image, labels = next(iter(train_loader))  # Get a batch of images and labels



image = image[5]

image = image.numpy().transpose((1, 2, 0))



# Convert the image to grayscale

gray_image = color.rgb2gray(image)



# Apply Local Binary Pattern

lbp_image = local_binary_pattern(gray_image, n_points, radius, method='uniform')



# Plot the original image and the LBP image

plt.figure(figsize=(12, 5))



plt.subplot(1, 2, 1)

plt.imshow(gray_image, cmap='gray')

plt.title('Original Image')

plt.axis('off')



plt.subplot(1, 2, 2)

plt.imshow(lbp_image, cmap='gray')

plt.title('Local Binary Pattern')

plt.axis('off')



plt.tight_layout()

plt.show()


In [ ]:
import numpy as np

import matplotlib.pyplot as plt

from skimage import color

from scipy.ndimage import generic_filter



radius = 1  # Radius of the circle

n_points = 8 * radius  # Number of points considered for the LBP



# Load a batch of 5 images

images, labels = next(iter(train_loader))  # Get a batch of images and labels



# Function to compute the mean, variance, and median-based LBP

def mvm_lbp(pixel_values):

    neighbors = pixel_values[:len(pixel_values) // 2]

    mean_val = np.mean(neighbors)

    variance_val = np.var(neighbors)

    median_val = np.median(neighbors)



    # Compute the LBP using mean, variance, and median as thresholds

    lbp_mean = np.sum((neighbors >= mean_val).astype(int) * (1 << np.arange(len(neighbors))))

    lbp_variance = np.sum((neighbors >= variance_val).astype(int) * (1 << np.arange(len(neighbors))))

    lbp_median = np.sum((neighbors >= median_val).astype(int) * (1 << np.arange(len(neighbors))))



    # Combine the results by averaging

    combined_lbp = (lbp_mean + lbp_variance + lbp_median) / 3

    return combined_lbp



# Define the size of the filter (3x3 for radius=1, 5x5 for radius=2, etc.)

filter_size = (2 * radius + 1, 2 * radius + 1)



# Set up the plot for 5 images

plt.figure(figsize=(15, 6))



for i in range(5):

    # Select the ith image from the batch and process it

    image = images[i].numpy().transpose((1, 2, 0))  # Convert to HxWxC format

    gray_image = color.rgb2gray(image)  # Convert to grayscale



    # Apply the MVM LBP filter to the grayscale image

    mvm_lbp_image = generic_filter(gray_image, mvm_lbp, size=filter_size)



    # Plot the original and the MVM LBP image side by side

    plt.subplot(2, 5, i + 1)

    plt.imshow(gray_image, cmap='gray')

    plt.title(f'Original Image {i+1}')

    plt.axis('off')



    plt.subplot(2, 5, i + 6)

    plt.imshow(mvm_lbp_image, cmap='gray')

    plt.title(f'MVM LBP Image {i+1}')

    plt.axis('off')



plt.tight_layout()

plt.show()
